In [1]:
%matplotlib inline

import numpy as np
import matplotlib.pyplot as plt
from IPython.display import display
from scipy.stats import multivariate_normal as mvn
from jax import config


import vsem_jax as vsem
from vsem_inv_prob import (
    InvProb, 
    VSEMPrior, 
    VSEMLikelihood,
    VSEMTest
)

import sys
sys.path.append("./../linear_Gaussian/")
from Gaussian import Gaussian

rng = np.random.default_rng(6435)
config.update("jax_enable_x64", True)


/opt/miniconda3/envs/modmcmc-dev/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
#
# Exact Inverse Problem
#

n_days = 365 * 2
par_names = ["kext", "av"]

# For exact MCMC
proposal_cov = np.diag([0.3**2, 0.04**2])

likelihood = VSEMLikelihood(rng, n_days, par_names)
prior = VSEMPrior(likelihood.par_names, rng)
inv_prob = InvProb(rng, prior, likelihood, proposal_cov=proposal_cov)

In [3]:
#
# Surrogate Inverse Problem
#

vsem_test = VSEMTest(inv_prob, n_design=10)

Optimization terminated successfully.
         Current function value: 40.903678
         Iterations: 74
         Function evaluations: 94
         Gradient evaluations: 94


In [ ]:
likelihood.plot_driver()
likelihood.plot_ground_truth()

In [ ]:
inv_prob.reset_sampler()
samp, trace_plot = inv_prob.sample_posterior(n_step=10000)

display(trace_plot)

In [ ]:
plt.hist2d(samp[:,0], samp[:,1], bins=30, cmap='Blues')
plt.plot(*likelihood.par_true, "*", color="red", markersize=12)
plt.colorbar(label='Counts per bin')
plt.xlabel('u1')
plt.ylabel('u2')
plt.title('Exact posterior')
plt.show()

In [4]:
fig_exact, ax_exact = vsem_test.plot_exact_log_post()
fig_gp_post, ax_gp_post = vsem_test.plot_gp_pred(conditional=True, latent_pred=True, figsize=(10,5))
fig_comparison, ax_comparison = vsem_test.plot_true_vs_gp_mean(conditional=True, latent_pred=True, figsize=(10,5))

/Users/andrewroberts/Desktop/git-repos/bip-surrogates-paper/experiments/vsem/vsem_inv_prob.py:768: UserWarning: This figure includes Axes that are not compatible with tight_layout, so results might be incorrect.
  fig.tight_layout()


In [5]:
ax_comparison.show()

AttributeError: 'numpy.ndarray' object has no attribute 'show'

In [ ]:
# Compute shared color limits
vmin = min(np.min(predictive_mean), np.min(log_post_grid))
vmax = max(np.max(predictive_mean), np.max(log_post_grid))

fig, axs = plt.subplots(1, 2, figsize=(10, 5))  # Wider and taller

axs = np.array(axs).reshape(-1)

# Plot 1: GP Predictive Mean
m1 = axs[0].pcolormesh(
    U1_grid,
    U2_grid,
    predictive_std.reshape(U1_grid.shape),
    shading='auto',
    cmap='viridis',
    vmin=vmin,
    vmax=vmax,
)
axs[0].plot(*likelihood.par_true, "*", color="red", markersize=12)
axs[0].plot(design.X[:,0], design.X[:,1], "o", color="red")
axs[0].set_title("GP Predictive Mean")
axs[0].set_xlabel("u1")
axs[0].set_ylabel("u2")

# Plot 2: Log Posterior
m2 = axs[1].pcolormesh(
    U1_grid,
    U2_grid,
    log_post_grid,
    shading='auto',
    cmap='viridis',
    vmin=vmin,
    vmax=vmax,
)
axs[1].plot(*likelihood.par_true, "*", color="red", markersize=12)
axs[1].plot(design.X[:,0], design.X[:,1], "o", color="red")
axs[1].set_title("True")
axs[1].set_xlabel("u1")
axs[1].set_ylabel("u2")

fig.tight_layout()



# Add horizontal colorbar beneath the plots
cbar = fig.colorbar(
    m1, ax=axs, orientation='horizontal',
    fraction=0.07, pad=0.18
)
cbar.set_label('Shared scale')

fig.show()

In [ ]:
vmin = min(np.min(predictive_std), np.min(log_post_grid))
vmax = max(np.max(predictive_std), np.max(log_post_grid))


fig, axs = plt.subplots(1, 2, figsize=(10, 5))

axs = np.array(axs).reshape(-1)

# --- First plot ---
m1 = axs[0].pcolormesh(
    U1_grid, U2_grid,
    predictive_std.reshape(U1_grid.shape),
    shading='auto', cmap='viridis'
)
axs[0].plot(*likelihood.par_true, "*", color="red", markersize=12)
axs[0].plot(design.X[:,0], design.X[:,1], "o", color="red")
axs[0].set_title("GP Predictive SD")
axs[0].set_xlabel("u1")
axs[0].set_ylabel("u2")

# --- Second plot ---
m2 = axs[1].pcolormesh(
    U1_grid, U2_grid,
    log_post_grid,
    shading='auto', cmap='viridis'
)
axs[1].plot(*likelihood.par_true, "*", color="red", markersize=12)
axs[1].plot(design.X[:,0], design.X[:,1], "o", color="red")
axs[1].set_title("True")
axs[1].set_xlabel("u1")
axs[1].set_ylabel("u2")

fig.tight_layout()
fig.subplots_adjust(bottom=0.28)  # extra space for both colorbars

# Plot colorbars beneath each plot, horizontally
cbar1 = fig.colorbar(
    m1, ax=axs[0], orientation='horizontal',
    fraction=0.07, pad=0.22
)
cbar1.set_label('GP Predictive SD scale')

cbar2 = fig.colorbar(
    m2, ax=axs[1], orientation='horizontal',
    fraction=0.07, pad=0.22
)
cbar2.set_label('Posterior scale')

fig.show()